In [71]:
import pandas as pd 
import numpy as np 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as FF
from wordcloud import WordCloud
import plotly.express as px

In [300]:
itinerarios= pd.read_parquet('../Data/Itinerarios/ItinerariosMerged.parquet')
bases=pd.read_csv('../Data/bases_def.csv')


In [301]:
itinerarios.shape

(1862981, 11)

In [302]:
cols_bases=['Número', 'Distrito', 'Barrio','Número de Plazas', 'Latitud', 'Longitud']
cols_salida=[palabra+'_Salida' for palabra in cols_bases]
cols_llegada=[palabra+'_Llegada' for palabra in cols_bases]

itinerarios_bases0=pd.merge(itinerarios, bases[cols_bases], how='left', left_on='idunplug_station', right_on='Número')
itinerarios_bases0.rename(columns=dict(zip(cols_bases, cols_salida)), inplace=True)
itinerarios_bases=pd.merge(itinerarios_bases0, bases[cols_bases], how='left', left_on='idplug_station', right_on='Número')
itinerarios_bases.rename(columns=dict(zip(cols_bases, cols_llegada)), inplace=True)
itinerarios_bases.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
0,{'$oid': '5ee862cecc8c1e839b419387'},509e6a72b7673f533912bf647d20df19305b848fbc1b89...,5,1,20,154,44,5,40,2019-07-01T00:00:00Z,...,01-02 EMBAJADORES,21.0,40.409816,-3.703914,40.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.413787,-3.704746
1,{'$oid': '5ee862cecc8c1e839b41938c'},90069a1194c3eb9e6faecaaf4eefb7209e890c0df5aeac...,8,1,17,184,57,4,129,2019-07-01T00:00:00Z,...,01-05 UNIVERSIDAD,24.0,40.429636,-3.712818,129.0,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438468,-3.698282
2,{'$oid': '5ee862cecc8c1e839b419394'},b0ecaab6a35b7ea44d118f851f7fad5c2237e7dd51b337...,15,1,16,282,53,4,43,2019-07-01T00:00:00Z,...,01-02 EMBAJADORES,24.0,40.408280,-3.700730,43.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.410089,-3.702545
3,{'$oid': '5ee862cecc8c1e839b41939e'},5e1b8a8c4603ee8ac4f06aba12ff51e2c5dd105b6e5193...,18,1,18,388,56,0,13,2019-07-01T00:00:00Z,...,01-06 SOL,24.0,40.418484,-3.703288,13.0,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427365,-3.710473
4,{'$oid': '5ee862cecc8c1e839b4193a5'},80974c4043db61721399c43f296ce1db4436c7f107f8fc...,1,1,20,254,105,0,75,2019-07-01T00:00:00Z,...,04-01 RECOLETOS,24.0,40.427068,-3.687474,75.0,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408339,-3.678415


In [303]:
itinerarios_bases.shape

(1862981, 23)

In [304]:
a=itinerarios_bases[itinerarios_bases['Número_Salida'].isna()].idunplug_station.value_counts()
for i in a.index: 
    print("-"+str(i)+"-")

-64-
-21-
-66-
-2008-


In [305]:
itinerarios_bases.drop(itinerarios_bases[itinerarios_bases['Número_Salida'].isna()].index, inplace=True)
itinerarios_bases.drop(itinerarios_bases[itinerarios_bases['Número_Llegada'].isna()].index, inplace=True)

itinerarios_bases.isna().sum()


_id                         0
user_day_code               0
idplug_base                 0
user_type                   0
idunplug_base               0
travel_time                 0
idunplug_station            0
ageRange                    0
idplug_station              0
unplug_hourTime             0
zip_code                    0
Número_Salida               0
Distrito_Salida             0
Barrio_Salida               0
Número de Plazas_Salida     0
Latitud_Salida              0
Longitud_Salida             0
Número_Llegada              0
Distrito_Llegada            0
Barrio_Llegada              0
Número de Plazas_Llegada    0
Latitud_Llegada             0
Longitud_Llegada            0
dtype: int64

In [306]:
itinerarios['idplug_station']=itinerarios['idplug_station'].astype('str')
itinerarios['idunplug_station']=itinerarios['idunplug_station'].astype('str')

itinerarios['Origen_destino']=itinerarios.apply(lambda x: x.idunplug_station+'-'+x.idplug_station, axis=1)


In [308]:
itinerarios.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code,Origen_destino
0,{'$oid': '5ee862cecc8c1e839b419387'},509e6a72b7673f533912bf647d20df19305b848fbc1b89...,5,1,20,154,44,5,40,2019-07-01T00:00:00Z,28007,44-40
1,{'$oid': '5ee862cecc8c1e839b41938c'},90069a1194c3eb9e6faecaaf4eefb7209e890c0df5aeac...,8,1,17,184,57,4,129,2019-07-01T00:00:00Z,28045,57-129
2,{'$oid': '5ee862cecc8c1e839b419394'},b0ecaab6a35b7ea44d118f851f7fad5c2237e7dd51b337...,15,1,16,282,53,4,43,2019-07-01T00:00:00Z,,53-43
3,{'$oid': '5ee862cecc8c1e839b41939e'},5e1b8a8c4603ee8ac4f06aba12ff51e2c5dd105b6e5193...,18,1,18,388,56,0,13,2019-07-01T00:00:00Z,,56-13
4,{'$oid': '5ee862cecc8c1e839b4193a5'},80974c4043db61721399c43f296ce1db4436c7f107f8fc...,1,1,20,254,105,0,75,2019-07-01T00:00:00Z,,105-75


In [309]:
len(itinerarios['idplug_station'].unique())


213

In [310]:
itinerarios.groupby('Origen_destino')['travel_time'].agg(['count', 'mean', 'max', 'min']).sort_values('count', ascending=False)

,count,mean,max,min
Origen_destino,,,,
9-149,1163,782.808255,21041,357
132-132,1162,2726.962995,19879,2
6-6,1133,2091.162401,21326,3
64-64,1101,2848.142598,18730,2
135-135,1057,2281.262062,41727,2
...,...,...,...,...
156-206,1,439.000000,439,439
127-117,1,1355.000000,1355,1355
205-117,1,1446.000000,1446,1446


In [311]:
itinerarios_bases.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,...,Barrio_Salida,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada
0,{'$oid': '5ee862cecc8c1e839b419387'},509e6a72b7673f533912bf647d20df19305b848fbc1b89...,5,1,20,154,44,5,40,2019-07-01T00:00:00Z,...,01-02 EMBAJADORES,21.0,40.409816,-3.703914,40.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.413787,-3.704746
1,{'$oid': '5ee862cecc8c1e839b41938c'},90069a1194c3eb9e6faecaaf4eefb7209e890c0df5aeac...,8,1,17,184,57,4,129,2019-07-01T00:00:00Z,...,01-05 UNIVERSIDAD,24.0,40.429636,-3.712818,129.0,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438468,-3.698282
2,{'$oid': '5ee862cecc8c1e839b419394'},b0ecaab6a35b7ea44d118f851f7fad5c2237e7dd51b337...,15,1,16,282,53,4,43,2019-07-01T00:00:00Z,...,01-02 EMBAJADORES,24.0,40.408280,-3.700730,43.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.410089,-3.702545
3,{'$oid': '5ee862cecc8c1e839b41939e'},5e1b8a8c4603ee8ac4f06aba12ff51e2c5dd105b6e5193...,18,1,18,388,56,0,13,2019-07-01T00:00:00Z,...,01-06 SOL,24.0,40.418484,-3.703288,13.0,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427365,-3.710473
4,{'$oid': '5ee862cecc8c1e839b4193a5'},80974c4043db61721399c43f296ce1db4436c7f107f8fc...,1,1,20,254,105,0,75,2019-07-01T00:00:00Z,...,04-01 RECOLETOS,24.0,40.427068,-3.687474,75.0,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408339,-3.678415


## WordCloud

In [312]:
itinerarios_bases=itinerarios_bases.astype({'Distrito_Llegada': str})
itinerarios_bases['Distrito_Llegada']=itinerarios_bases['Distrito_Llegada'].apply(lambda x: x.replace('\xa0', ''))
itinerarios_bases['Distrito_Llegada'].unique()

array(['01 CENTRO', '07 CHAMBERÍ', '03 RETIRO', '05 CHAMARTÍN',
       '04 SALAMANCA', '02 ARGANZUELA', '06 TETUÁN', '09 MONCLOA-ARAVACA',
       '12 USERA', '13 PUENTE DE VALLECAS', '15 CIUDAD LINEAL',
       '14 MORATALAZ', '11 CARABANCHEL', '10 LATINA',
       '08 FUENCARRAL-EL PARDO'], dtype=object)

In [313]:
from collections import Counter 
a=Counter(barrios_llegada_wordcloud)
a

Counter({' CENTRO': 569400,
         ' CHAMBERÍ': 228602,
         ' RETIRO': 230053,
         ' CHAMARTÍN': 142026,
         ' SALAMANCA': 289072,
         ' ARGANZUELA': 232091,
         ' TETUÁN': 77390,
         ' MONCLOA-ARAVACA': 96747,
         ' USERA': 9206,
         ' PUENTE DE VALLECAS': 7861,
         ' CIUDAD LINEAL': 19925,
         ' MORATALAZ': 11036,
         ' CARABANCHEL': 6362,
         ' LATINA': 2734,
         ' FUENCARRAL-EL PARDO': 2677})

In [314]:
import matplotlib.pyplot as plt
barrios_llegada = itinerarios_bases['Distrito_Llegada']
numbers=list(range(10))

barrios_llegada_wordcloud=["".join(i for i in palabra if not i.isdigit()) for palabra in barrios_llegada]
barrios_frec=Counter(barrios_llegada_wordcloud)

set(barrios_llegada_wordcloud)


wordcloud = WordCloud (
                    background_color = 'white',
                    width = 1000,
                    height = 750,
                    collocations=False).generate_from_frequencies(barrios_frec)

fig=px.imshow(wordcloud) # image show
fig.update_layout(width=1000, height=750)
fig.show()


## Mapa Rutas

In [315]:
itinerarios_bases['idplug_station']=itinerarios_bases['idplug_station'].astype('str')
itinerarios_bases['idunplug_station']=itinerarios_bases['idunplug_station'].astype('str')

itinerarios_bases['Origen_destino']=itinerarios_bases.apply(lambda x: x.idunplug_station+'-'+x.idplug_station, axis=1)



In [316]:
itinerarios_bases.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,...,Número de Plazas_Salida,Latitud_Salida,Longitud_Salida,Número_Llegada,Distrito_Llegada,Barrio_Llegada,Número de Plazas_Llegada,Latitud_Llegada,Longitud_Llegada,Origen_destino
0,{'$oid': '5ee862cecc8c1e839b419387'},509e6a72b7673f533912bf647d20df19305b848fbc1b89...,5,1,20,154,44,5,40,2019-07-01T00:00:00Z,...,21.0,40.409816,-3.703914,40.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.413787,-3.704746,44-40
1,{'$oid': '5ee862cecc8c1e839b41938c'},90069a1194c3eb9e6faecaaf4eefb7209e890c0df5aeac...,8,1,17,184,57,4,129,2019-07-01T00:00:00Z,...,24.0,40.429636,-3.712818,129.0,07 CHAMBERÍ,07-05 RÍOS ROSAS,24.0,40.438468,-3.698282,57-129
2,{'$oid': '5ee862cecc8c1e839b419394'},b0ecaab6a35b7ea44d118f851f7fad5c2237e7dd51b337...,15,1,16,282,53,4,43,2019-07-01T00:00:00Z,...,24.0,40.408280,-3.700730,43.0,01 CENTRO,01-02 EMBAJADORES,24.0,40.410089,-3.702545,53-43
3,{'$oid': '5ee862cecc8c1e839b41939e'},5e1b8a8c4603ee8ac4f06aba12ff51e2c5dd105b6e5193...,18,1,18,388,56,0,13,2019-07-01T00:00:00Z,...,24.0,40.418484,-3.703288,13.0,01 CENTRO,01-05 UNIVERSIDAD,24.0,40.427365,-3.710473,56-13
4,{'$oid': '5ee862cecc8c1e839b4193a5'},80974c4043db61721399c43f296ce1db4436c7f107f8fc...,1,1,20,254,105,0,75,2019-07-01T00:00:00Z,...,24.0,40.427068,-3.687474,75.0,03 RETIRO,03-05 JERÓNIMOS,24.0,40.408339,-3.678415,105-75


In [277]:
df_rutas.groupby('Origen_destino').count().sort_values('_id', ascending=False)

,Latitud_Salida,Longitud_Salida,Distrito_Salida,Latitud_Llegada,Longitud_Llegada,Distrito_Llegada,idplug_station,idunplug_station,_id
Origen_destino,,,,,,,,,
90-140,4,4,4,4,4,4,4,4,4
128-90,4,4,4,4,4,4,4,4,4
140-90,4,4,4,4,4,4,4,4,4
161-90,4,4,4,4,4,4,4,4,4
128-20,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...
164-149,1,1,1,1,1,1,1,1,1
164-15,1,1,1,1,1,1,1,1,1
164-150,1,1,1,1,1,1,1,1,1


In [278]:
df_rutas[df_rutas['Origen_destino']=='90-140']

,Origen_destino,Latitud_Salida,Longitud_Salida,Distrito_Salida,Latitud_Llegada,Longitud_Llegada,Distrito_Llegada,idplug_station,idunplug_station,_id
42460,90-140,40.425088,-3.683730,04 SALAMANCA,40.438520,-3.682913,05 CHAMARTÍN,140,90,21
42461,90-140,40.425088,-3.683730,04 SALAMANCA,40.438628,-3.682904,05 CHAMARTÍN,140,90,21
42459,90-140,40.425079,-3.683561,04 SALAMANCA,40.438628,-3.682904,05 CHAMARTÍN,140,90,21
42458,90-140,40.425079,-3.683561,04 SALAMANCA,40.438520,-3.682913,05 CHAMARTÍN,140,90,21


In [317]:
cols_rutas=['Origen_destino','Latitud_Salida','Longitud_Salida','Distrito_Salida','Latitud_Llegada','Longitud_Llegada','Distrito_Llegada', 'idplug_station', 'idunplug_station' ]
df_rutas=itinerarios_bases[itinerarios_bases['idplug_station']!=itinerarios_bases['idunplug_station'] ].groupby(cols_rutas)['_id'].count().to_frame().reset_index().sort_values('_id', ascending=False)

top10rutas=df_rutas.head(16)
top10rutas['ruta']=top10rutas.apply(lambda x: sorted([x.Longitud_Salida, x.Latitud_Salida, x.Longitud_Llegada, x.Latitud_Llegada]), axis=1)
top10rutas['ruta']=top10rutas['ruta'].apply(lambda x: ' '.join([str(word) for word in x]))
cols_rutas=['Latitud_Salida','Longitud_Salida','Latitud_Llegada','Longitud_Llegada', 'ruta']
def rutas(df): 
    
    
    long_llegada=df['Longitud_Llegada'].iloc[0]
    lat_llegada=df['Latitud_Llegada'].iloc[0]
    long_salida=df['Longitud_Salida'].iloc[0]
    lat_salida=df['Latitud_Salida'].iloc[0]
    _id=df['_id'].sum()
    cols= ['Latitud_Salida','Longitud_Salida','Latitud_Llegada','Longitud_Llegada', 'viajes']
    datos=[lat_salida, long_salida,lat_llegada, long_llegada, _id ]
    return pd.Series(dict(zip(cols,datos)))
             
top7rutas= top10rutas.groupby('ruta').apply(rutas).reset_index()
top7rutas

,ruta,Latitud_Salida,Longitud_Salida,Latitud_Llegada,Longitud_Llegada,viajes
0,-3.6983033 -3.6928901 40.3962687 40.4414845,40.441485,-3.692890,40.396269,-3.698303,1277.0
1,-3.6983033 -3.6955803 40.3962687 40.4132831,40.396269,-3.698303,40.413283,-3.695580,1528.0
2,-3.6992427 -3.6928901 40.4060497 40.4414845,40.441485,-3.692890,40.406050,-3.699243,1270.0
3,-3.7017071 -3.6878512 40.4417195 40.4498448,40.441719,-3.701707,40.449845,-3.687851,1424.0
4,-3.7017071 -3.6893305 40.4417195 40.459282,40.459282,-3.689331,40.441719,-3.701707,613.0
5,-3.7025446 -3.6983033 40.3962687 40.4100889,40.396269,-3.698303,40.410089,-3.702545,1385.0
6,-3.7035911 -3.6983033 40.3962687 40.4463889,40.446389,-3.703591,40.396269,-3.698303,769.0
7,-3.7094093 -3.6878512 40.4156134 40.4498448,40.415613,-3.709409,40.449845,-3.687851,1940.0
8,-3.7094093 -3.6893305 40.4156134 40.459282,40.415613,-3.709409,40.459282,-3.689331,615.0
9,-3.7128181 -3.7114443 40.4112961 40.4296363,40.429636,-3.712818,40.411296,-3.711444,656.0


In [318]:
cols_llegada_estacion=['Longitud_Llegada','Latitud_Llegada', 'Distrito_Llegada' ]
cols_salida_estacion=['Longitud_Salida', 'Latitud_Salida','Distrito_Salida']
cols_estaciones=['Longitud','Latitud', 'Distrito']

estaciones_llegada=top10rutas[cols_llegada_estacion]
estaciones_salida=top10rutas[cols_salida_estacion]
estaciones_llegada.rename(columns=dict(zip(cols_llegada_estacion,cols_estaciones)) , inplace=True)
estaciones_salida.rename(columns=dict(zip(cols_salida_estacion,cols_estaciones)) , inplace=True)

top10estaciones0=pd.concat([estaciones_salida, estaciones_llegada], axis=0, ignore_index=True )
top10estaciones0['count']=1
def top_estaciones(df): 
    distrito= df['Distrito'].iloc[0]
    count=df['count'].sum()
    cols=['Distrito', 'count']
    datos=[distrito, count]
    return(pd.Series(dict(zip(cols,datos))))
top10estaciones=top10estaciones0.groupby(['Longitud','Latitud']).apply(top_estaciones).reset_index()

top10estaciones

,Longitud,Latitud,Distrito,count
0,-3.712818,40.429636,01 CENTRO,1
1,-3.711444,40.411296,01 CENTRO,1
2,-3.709409,40.415613,01 CENTRO,3
3,-3.703591,40.446389,07 CHAMBERÍ,1
4,-3.702545,40.410089,01 CENTRO,2
5,-3.701707,40.441719,07 CHAMBERÍ,3
6,-3.699243,40.406050,01 CENTRO,2
7,-3.698303,40.396269,02 ARGANZUELA,7
8,-3.695580,40.413283,01 CENTRO,2
9,-3.692890,40.441485,07 CHAMBERÍ,4


In [319]:

print(top10estaciones)
#fig = go.Figure()
fig = px.scatter_mapbox(top10estaciones, lat="Latitud", lon="Longitud", size='count',color='Distrito', width = 1000, height = 900, zoom = 50)
# fig.add_trace(go.Scattermapbox(
#         lat = top10estaciones["Latitud"],
#         lon=top10estaciones["Longitud"],
#         mode='markers',
#         showlegend=False,
#         marker=go.scattermapbox.Marker(
#             size=top10estaciones["count"],
#             color='blue', 
#             opacity=0.7,
#             #size=50,
#              )
#     ))
for i in range(top7rutas.shape[0]): 
    valores=top7rutas.iloc[i,:]

    fig.add_trace(
        go.Scattermapbox(
        mode = "markers+lines",
        lon = [valores['Longitud_Salida'], valores['Longitud_Llegada']],
        lat = [valores['Latitud_Salida'], valores['Latitud_Llegada']],
        line= {'width':valores['viajes']*0.005} ,
        name= 'Ruta '+str(1+i)

        ))
hover_header = 'Estacion: %{text} '

fig.update_layout(
    title='Rutas más concurridas',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    width = 1000,
    height = 1000,
    mapbox=dict(
        bearing=0,
        center=dict(
            lat=40.435,
            lon=-3.69
        ),
        zoom=12.5,
        style= 'carto-positron' # 'open-street-map'
        
    )
)
fig.show()

    Longitud    Latitud        Distrito  count
0  -3.712818  40.429636      01  CENTRO      1
1  -3.711444  40.411296       01 CENTRO      1
2  -3.709409  40.415613      01  CENTRO      3
3  -3.703591  40.446389    07  CHAMBERÍ      1
4  -3.702545  40.410089      01  CENTRO      2
5  -3.701707  40.441719    07  CHAMBERÍ      3
6  -3.699243  40.406050      01  CENTRO      2
7  -3.698303  40.396269  02  ARGANZUELA      7
8  -3.695580  40.413283      01  CENTRO      2
9  -3.692890  40.441485    07  CHAMBERÍ      4
10 -3.689331  40.459282   05  CHAMARTÍN      2
11 -3.687851  40.449845   05  CHAMARTÍN      4


### Mapa rutas desde una estación en concreto

In [320]:
df_rutas.head()

,Origen_destino,Latitud_Salida,Longitud_Salida,Distrito_Salida,Latitud_Llegada,Longitud_Llegada,Distrito_Llegada,idplug_station,idunplug_station,_id
40380,9-149,40.415613,-3.709409,01 CENTRO,40.449845,-3.687851,05 CHAMARTÍN,149,9,1163
17324,175-27,40.396269,-3.698303,02 ARGANZUELA,40.413283,-3.695580,01 CENTRO,27,175,851
11462,149-9,40.449845,-3.687851,05 CHAMARTÍN,40.415613,-3.709409,01 CENTRO,9,149,777
7663,132-175,40.446389,-3.703591,07 CHAMBERÍ,40.396269,-3.698303,02 ARGANZUELA,175,132,769
17341,175-43,40.396269,-3.698303,02 ARGANZUELA,40.410089,-3.702545,01 CENTRO,43,175,754


In [321]:
df_rutas.groupby('idplug_station')['_id'].sum().sort_values(ascending=False)

idplug_station
175    25268
163    22227
43     21718
57     20627
83     19032
       ...  
16      2154
197     2115
203     1713
209     1583
28      1272
Name: _id, Length: 209, dtype: int64

In [325]:
rutas_175=df_rutas[df_rutas['idplug_station']== '175']
rutas_175_top=rutas_175.head(10)

In [330]:
rutas_175_top.rename(columns={'_id': 'viajes'}, inplace=True)

In [327]:
cols_llegada_estacion=['Longitud_Llegada','Latitud_Llegada', 'Distrito_Llegada' ]
cols_salida_estacion=['Longitud_Salida', 'Latitud_Salida','Distrito_Salida']
cols_estaciones=['Longitud','Latitud', 'Distrito']

estaciones_llegada=rutas_175_top[cols_llegada_estacion]
estaciones_salida=rutas_175_top[cols_salida_estacion]
estaciones_llegada.rename(columns=dict(zip(cols_llegada_estacion,cols_estaciones)) , inplace=True)
estaciones_salida.rename(columns=dict(zip(cols_salida_estacion,cols_estaciones)) , inplace=True)

top175estaciones0=pd.concat([estaciones_salida, estaciones_llegada], axis=0, ignore_index=True )
top175estaciones0['count']=1
def top_estaciones(df): 
    distrito= df['Distrito'].iloc[0]
    count=df['count'].sum()
    cols=['Distrito', 'count']
    datos=[distrito, count]
    return(pd.Series(dict(zip(cols,datos))))
top175estaciones=top175estaciones0.groupby(['Longitud','Latitud']).apply(top_estaciones).reset_index()

top175estaciones

,Longitud,Latitud,Distrito,count
0,-3.711444,40.411296,01 CENTRO,1
1,-3.711138,40.406949,01 CENTRO,1
2,-3.703591,40.446389,07 CHAMBERÍ,1
3,-3.703288,40.418484,01 CENTRO,1
4,-3.702545,40.410089,01 CENTRO,1
5,-3.701781,40.417162,01 CENTRO,1
6,-3.700784,40.414518,01 CENTRO,1
7,-3.698303,40.396269,02 ARGANZUELA,10
8,-3.695580,40.413283,01 CENTRO,1
9,-3.694542,40.423831,01 CENTRO,1


In [331]:

print(top175estaciones)
#fig = go.Figure()
fig = px.scatter_mapbox(top175estaciones, lat="Latitud", lon="Longitud", size='count',color='Distrito', width = 1000, height = 900, zoom = 50)
# fig.add_trace(go.Scattermapbox(
#         lat = top10estaciones["Latitud"],
#         lon=top10estaciones["Longitud"],
#         mode='markers',
#         showlegend=False,
#         marker=go.scattermapbox.Marker(
#             size=top10estaciones["count"],
#             color='blue', 
#             opacity=0.7,
#             #size=50,
#              )
#     ))
for i in range(rutas_175_top.shape[0]): 
    valores=rutas_175_top.iloc[i,:]

    fig.add_trace(
        go.Scattermapbox(
        mode = "markers+lines",
        lon = [valores['Longitud_Salida'], valores['Longitud_Llegada']],
        lat = [valores['Latitud_Salida'], valores['Latitud_Llegada']],
        line= {'width':valores['viajes']*0.005} ,
        name= 'Ruta '+str(1+i)

        ))
hover_header = 'Estacion: %{text} '

fig.update_layout(
    title='Rutas más concurridas',
    autosize=True,
    hovermode='closest',
    showlegend=True,
    width = 1000,
    height = 1000,
    mapbox=dict(
        bearing=0,
        center=dict(
            lat=40.435,
            lon=-3.69
        ),
        zoom=12.5,
        style= 'carto-positron' # 'open-street-map'
        
    )
)
fig.show()

    Longitud    Latitud       Distrito  count
0  -3.711444  40.411296     01  CENTRO      1
1  -3.711138  40.406949     01  CENTRO      1
2  -3.703591  40.446389   07  CHAMBERÍ      1
3  -3.703288  40.418484     01  CENTRO      1
4  -3.702545  40.410089     01  CENTRO      1
5  -3.701781  40.417162     01  CENTRO      1
6  -3.700784  40.414518     01  CENTRO      1
7  -3.698303  40.396269  02 ARGANZUELA     10
8  -3.695580  40.413283     01  CENTRO      1
9  -3.694542  40.423831     01  CENTRO      1
10 -3.692890  40.441485   07  CHAMBERÍ      1
